In [ ]:
import os
import openai
import anthropic
import together
import subprocess
import re

# open ai API key
openai_api_key=''
# anthropic API key
claude_api_key = ''
# together AI API key
together_api_key = ''

In [ ]:
# read the prompt from the prompt text file titled "mm1queue.txt"
with open("../prompts/mm1queue.txt", "r") as f:
    prompt = f.read()

prompt = str(prompt)
print(prompt)

In [5]:
claude_client = anthropic.Anthropic(api_key = claude_api_key)
openai_client = openai.OpenAI(api_key = openai_api_key)
togetherai_client = together.Together(api_key=together_api_key)

## GPT 4o

In [4]:
completion = openai_client.chat.completions.create(
    model="gpt-4o",
    messages=[
        {"role": "system", "content": "You are an expert in probability theory and stochastic modeling."},
        {"role": "user", "content": prompt},
    ]
)

# get the response
response = completion.choices[0].message.content

In [5]:
print(response)

To tackle the problem of minimizing \( E[\text{avg\_sojourn\_time}] + c \times (\mu^2) \), we can use both analytical and simulation-based approaches. Here's a detailed explanation of both methods.

### Analytical Approach

In an M/M/1 queueing system, we know the average sojourn time (the time a customer spends in the system) is given by:

\[
E[\text{avg\_sojourn\_time}] = \frac{1}{\mu - \lambda}
\]

Given the penalty term \( c \times (\mu^2) \), we aim to minimize the following objective function:

\[
f(\mu) = \frac{1}{\mu - \lambda} + c \times (\mu^2)
\]

**Objective: Minimize \( f(\mu) \) subject to \( \mu \geq \lambda \).**

**Derivative and Critical Points:**

1. Compute the derivative of \( f(\mu) \) with respect to \( \mu\):

\[
f'(\mu) = -\frac{1}{(\mu - \lambda)^2} + 2c\mu
\]

2. Set the derivative to zero for critical points:

\[
-\frac{1}{(\mu - \lambda)^2} + 2c\mu = 0
\]

3. Solving this equation gives:

\[
2c\mu(\mu - \lambda)^2 = 1
\]

Find \( \mu \) that satisfies this 

In [7]:
import numpy as np

def simulate_queue(lambda_, mu, warmup, people):
    interarrival_times = np.random.exponential(1/lambda_, warmup + people)
    service_times = np.random.exponential(1/mu, warmup + people)
    
    arrival_times = np.cumsum(interarrival_times)
    service_start_times = np.zeros(warmup + people)
    service_end_times = np.zeros(warmup + people)
    wait_times = np.zeros(warmup + people)
    
    service_end_times[0] = arrival_times[0] + service_times[0]
    
    for i in range(1, warmup + people):
        service_start_times[i] = max(arrival_times[i], service_end_times[i-1])
        service_end_times[i] = service_start_times[i] + service_times[i]
        wait_times[i] = service_start_times[i] - arrival_times[i]
    
    sojourn_times = service_end_times - arrival_times
    
    # Exclude warmup
    sojourn_times = sojourn_times[warmup:]
    wait_times = wait_times[warmup:]
    
    avg_sojourn_time = np.mean(sojourn_times)
    avg_waiting_time = np.mean(wait_times)
    frac_cust_wait = np.mean(wait_times > 0)
    
    return avg_sojourn_time, avg_waiting_time, frac_cust_wait

def evaluate_objective(lambda_, mu, c, warmup, people):
    avg_sojourn_time, _, _ = simulate_queue(lambda_, mu, warmup, people)
    objective_value = avg_sojourn_time + c * (mu**2)
    return objective_value

# Example usage
lambda_ = 1.5
c = 0.1
warmup = 50
people = 200
mu_values = np.linspace(lambda_ + 0.01, 4.0, num=50)  # Search space for mu
objective_values = [evaluate_objective(lambda_, mu, c, warmup, people) for mu in mu_values]

optimal_mu = mu_values[np.argmin(objective_values)]
optimal_value = min(objective_values)

print(f"Optimal mu: {optimal_mu}")
print(f"Objective function value: {optimal_value}")

# run this 1000 times and find the mean and std deviation for the optimal value
num_trials = 1000
optimal_values = []
for _ in range(num_trials):
    objective_values = [evaluate_objective(lambda_, mu, c, warmup, people) for mu in mu_values]
    optimal_value = min(objective_values)
    optimal_values.append(optimal_value)
mean_optimal_value = np.mean(optimal_values)
std_optimal_value = np.std(optimal_values)
print(f"Mean of optimal values over {num_trials} trials: {mean_optimal_value}")
print(f"Standard deviation of optimal values over {num_trials} trials: {std_optimal_value}")

Optimal mu: 3.0344897959183674
Objective function value: 1.3925941863574987
Mean of optimal values over 1000 trials: 1.301083764138901
Standard deviation of optimal values over 1000 trials: 0.06235241872423378


## o1

In [8]:
completion = openai_client.chat.completions.create(
    model="o1",
    messages=[
        {"role": "system", "content": "You are an expert in probability theory and stochastic modeling."},
        {"role": "user", "content": prompt},
    ]
)

# get the response
response = completion.choices[0].message.content

In [9]:
print(response)

Below is a self‐contained discussion of how one can tackle the problem analytically (via approximate queueing formulas) and/or by simulation. A short Python code example is also provided. In the end, you will see how to arrive at a recommended service rate μ that balances low sojourn times against the penalty c⋅(μ²).

────────────────────────────────────────────────────────────────────────
1) ANALYTICAL / CLOSED‐FORM APPROACH
────────────────────────────────────────────────────────────────────────

1.1) The M/M/1 Setting

We have a single‐server queue with:
• Poisson arrivals at rate λ.  
• Exponential service times at rate μ (to be determined).  
• Stability condition requires μ > λ (or else mean queue length and sojourn times blow up).  

In an M/M/1 queue (stable, i.e. μ > λ), the average (theoretical) sojourn time E[T] is:
  
  E[T] = 1 / (μ − λ).

(Strictly, this arises from Little’s Law and the known M/M/1 formula. Note that if μ ≤ λ, the average time in the system diverges to in

In [1]:

import math
import random

def mm1_sim(lmbda, mu, warmup=50, people=200, seed=None):
    """
    Simulate an M/M/1 queue for 'warmup + people' customers.
    Returns the average sojourn time (after warmup), average waiting time, 
    fraction who waited, plus the final objective = sojourn + c*(mu^2) if c is given.
    We do not fix 'c' here; we just return sojourn so the caller can combine with c*(mu^2).
    """
    if seed is not None:
        random.seed(seed)

    # Arrival and service events
    t = 0.0   # current time
    arrival_times = []
    service_begin_times = []
    departure_times = []

    # Generate arrival times
    for i in range(warmup + people):
        # Interarrival time ~ Exp(lmbda)
        interarrival = random.expovariate(lmbda)
        t += interarrival
        arrival_times.append(t)
    
    # We will track the time at which the server becomes free
    server_available_time = 0.0

    for i in range(warmup + people):
        # If the server is free earlier than arrival time, the customer starts immediately
        start_service = max(server_available_time, arrival_times[i])
        service_begin_times.append(start_service)
        # Service time ~ Exp(mu)
        st = random.expovariate(mu)
        depart = start_service + st
        departure_times.append(depart)
        server_available_time = depart  # server next available after this departure
    
    # Compute sojourn and waiting times for each customer
    sojourn_times = []
    waiting_times = []
    for i in range(warmup + people):
        w = service_begin_times[i] - arrival_times[i]
        t_sys = departure_times[i] - arrival_times[i]
        sojourn_times.append(t_sys)
        waiting_times.append(w)
    
    # Discard warmup customers
    s_after = sojourn_times[warmup:]
    w_after = waiting_times[warmup:]
    
    avg_sojourn_time = sum(s_after) / len(s_after)
    avg_waiting_time = sum(w_after) / len(w_after)
    frac_cust_wait   = sum(1 for w_ in w_after if w_ > 0.0) / len(w_after)

    return avg_sojourn_time, avg_waiting_time, frac_cust_wait

def simulation_objective(lmbda, mu, c, warmup=50, people=200, replicates=1, seed=None):
    """
    Run replicates for a given mu, return average objective = average sojourn + c*(mu^2).
    """
    sojourn_sum = 0.0
    # We can vary the seed for each replicate if desired
    for r in range(replicates):
        this_seed = seed + r if seed is not None else None
        soj, _, _ = mm1_sim(lmbda, mu, warmup, people, seed=this_seed)[0:3]
        sojourn_sum += soj
    avg_sojourn = sojourn_sum / replicates
    return avg_sojourn + c*(mu**2)

def simple_search_for_mu(lmbda=1.5, c=0.1, budget=10, warmup=50, people=200, seed=42):
    """
    Demonstrate a simple search (trying a few candidate mu values).
    'budget' means how many mu values we test.
    """
    random.seed(seed)
    
    mu_candidates = []
    # Generate some random candidate mus, ensuring mu > lmbda
    for _ in range(budget):
        # We'll sample from something like Exp(1/3) + lambda as an example random approach:
        candidate = lmbda + random.expovariate(1.0/3.0)
        mu_candidates.append(candidate)
    
    # Also include the starting solution mu=3.0 explicitly:
    mu_candidates.append(3.0)
    
    best_mu = None
    best_obj = float('inf')
    
    for m in mu_candidates:
        # For speed in this demo, we do just 1 replicate per candidate.
        # In practice, you might do more replicates to reduce variability.
        obj_value = simulation_objective(lmbda, m, c, warmup, people, replicates=1, seed=seed)
        if obj_value < best_obj:
            best_obj = obj_value
            best_mu = m
    
    return best_mu, best_obj

if __name__ == "__main__":
    # Example usage:
    lmbda = 1.5
    c = 0.1
    budget = 1000  # number of different mu values to try in simple search
    
    mu_star, obj_est = simple_search_for_mu(lmbda, c, budget=budget, warmup=50, people=200)
    
    # Get separate stats from a final simulation with the best mu found:
    soj, w, fwait = mm1_sim(lmbda=lmbda, mu=mu_star, warmup=50, people=200, seed=123)
    penalty = c * (mu_star**2)
    total_objective = soj + penalty
    
    print(f"Best mu found via simple search: {mu_star:.4f}")
    print(f"  Estimated avg sojourn time = {soj:.4f}")
    print(f"  Penalty c * mu^2 = {penalty:.4f}")
    print(f"  Total objective = {total_objective:.4f}")
    print(f"  Avg waiting time = {w:.4f}, Fraction who waited = {fwait:.4f}")

# Note: The above code is a complete simulation and optimization for the M/M/1 queue problem.
    # run this 1000 times and find the mean and std deviation for the total objective
num_trials = 1000
total_objectives = []
for _ in range(num_trials):
    mu_star, obj_est = simple_search_for_mu(lmbda, c, budget=budget, warmup=50, people=200)
    soj, w, fwait = mm1_sim(lmbda=lmbda, mu=mu_star, warmup=50, people=200, seed=_)
    penalty = c * (mu_star**2)
    total_objective = soj + penalty
    total_objectives.append(total_objective)
mean_total_objective = np.mean(total_objectives)
std_total_objective = np.std(total_objectives)
print(f"Mean of total objectives over {num_trials} trials: {mean_total_objective:.4f}")
print(f"Standard deviation of total objectives over {num_trials} trials: {std_total_objective:.4f}")

Best mu found via simple search: 2.6873
  Estimated avg sojourn time = 0.7098
  Penalty c * mu^2 = 0.7222
  Total objective = 1.4320
  Avg waiting time = 0.3507, Fraction who waited = 0.4400
Mean of total objectives over 1000 trials: 1.5676
Standard deviation of total objectives over 1000 trials: 0.2097


## o3-mini

In [13]:
completion = client.chat.completions.create(
    model="o3-mini",
    messages=[
        {"role": "system", "content": "You are an expert in probability theory and stochastic modeling."},
        {"role": "user", "content": prompt},
    ]
)

In [14]:
response = completion.choices[0].message.content

In [15]:
import re
pattern = r'```python(.*?)```'
matches = re.findall(pattern, response, re.DOTALL)
if matches:
    code = matches[0].strip()
    print(code)
else:
    print("No code found in the response.")

import numpy as np

def simulate_mm1(mu, lam=1.5, warmup=50, people=200, seed=None):
    """
    Simulate an M/M/1 queue, given:
      - mu: service rate,
      - lam: arrival rate,
      - warmup: number of initial customers to discard,
      - people: number of customers for stats.
    Returns: average sojourn time, average waiting time, fraction of customers who waited.
    """
    if seed is not None:
        np.random.seed(seed)
        
    total_customers = warmup + people

    # Generate the interarrival times and arrival times for all customers.
    inter_arrivals = np.random.exponential(scale=1/lam, size=total_customers)
    arrivals = np.cumsum(inter_arrivals)
    
    # Generate service times.
    service_times = np.random.exponential(scale=1/mu, size=total_customers)
    
    # Initialize arrays for service start and end times.
    start_service = np.empty(total_customers)
    departure = np.empty(total_customers)
    
    for i in range(total_customers):
        if i == 0

In [16]:
subprocess.run(["python3", "-c", code])

Baseline (mu = 3.00):
  Simulated Average Sojourn Time = 0.6454
  Penalty c*(mu^2)                = 0.9000
  Total Objective                 = 1.5454
  Avg Waiting Time               = 0.2949
  Fraction of Customers Waiting  = 0.4600

Candidate (mu = 2.83):
  Simulated Average Sojourn Time = 0.7157
  Penalty c*(mu^2)                = 0.8009
  Total Objective                 = 1.5166
  Avg Waiting Time               = 0.3441
  Fraction of Customers Waiting  = 0.5000


CompletedProcess(args=['python3', '-c', 'import numpy as np\n\ndef simulate_mm1(mu, lam=1.5, warmup=50, people=200, seed=None):\n    """\n    Simulate an M/M/1 queue, given:\n      - mu: service rate,\n      - lam: arrival rate,\n      - warmup: number of initial customers to discard,\n      - people: number of customers for stats.\n    Returns: average sojourn time, average waiting time, fraction of customers who waited.\n    """\n    if seed is not None:\n        np.random.seed(seed)\n        \n    total_customers = warmup + people\n\n    # Generate the interarrival times and arrival times for all customers.\n    inter_arrivals = np.random.exponential(scale=1/lam, size=total_customers)\n    arrivals = np.cumsum(inter_arrivals)\n    \n    # Generate service times.\n    service_times = np.random.exponential(scale=1/mu, size=total_customers)\n    \n    # Initialize arrays for service start and end times.\n    start_service = np.empty(total_customers)\n    departure = np.empty(total_cust

## Claude 3.5 Sonnet

In [6]:
response = claude_client.messages.create(
    model="claude-3-5-sonnet-20241022",  # Or another Claude 3 model
    max_tokens=4096,
    system="You are an expert in probability theory and stochastic modeling.",
    messages=[
        {"role": "user", "content": prompt}
    ]
)

print(response.content[0].text)

I'll solve this problem using both analytical and simulation approaches.

1) ANALYTICAL APPROACH
For an M/M/1 queue, we have closed-form expressions:
- Average sojourn time = 1/(μ-λ)
- System must be stable: μ > λ

The objective function is:
f(μ) = 1/(μ-λ) + c*μ²

To find the minimum, we differentiate f(μ) with respect to μ and set to zero:
f'(μ) = -1/(μ-λ)² + 2cμ = 0

This gives us:
1/(μ-λ)² = 2cμ
1 = 2cμ(μ-λ)²
1 = 2cμ³ - 4cλμ² + 2cλ²μ

This cubic equation can be solved numerically.

2) SIMULATION-BASED APPROACH
I'll implement both approaches, using gradient descent for the simulation optimization:

```python
import numpy as np
from collections import deque
import heapq

def simulate_queue(mu, lambda_=1.5, warmup=50, people=200):
    np.random.seed(42)  # for reproducibility
    
    # Generate arrival and service times
    interarrival_times = np.random.exponential(1/lambda_, warmup + people)
    service_times = np.random.exponential(1/mu, warmup + people)
    
    arrival_times = np

In [8]:
import numpy as np
from collections import deque
import heapq

def simulate_queue(mu, lambda_=1.5, warmup=50, people=200):
    np.random.seed(42)  # for reproducibility
    
    # Generate arrival and service times
    interarrival_times = np.random.exponential(1/lambda_, warmup + people)
    service_times = np.random.exponential(1/mu, warmup + people)
    
    arrival_times = np.cumsum(interarrival_times)
    
    # Simulation variables
    waiting_times = []
    sojourn_times = []
    server_busy_until = 0
    
    # Process each customer
    for i in range(warmup + people):
        arrival = arrival_times[i]
        service = service_times[i]
        
        # Calculate waiting and sojourn times
        wait = max(0, server_busy_until - arrival)
        service_start = arrival + wait
        departure = service_start + service
        
        server_busy_until = departure
        
        if i >= warmup:  # Only collect statistics after warmup
            waiting_times.append(wait)
            sojourn_times.append(departure - arrival)
    
    avg_sojourn = np.mean(sojourn_times)
    avg_waiting = np.mean(waiting_times)
    frac_wait = np.mean([w > 0 for w in waiting_times])
    
    return avg_sojourn, avg_waiting, frac_wait

def objective(mu, c=0.1, lambda_=1.5):
    if mu <= lambda_:  # Unstable system
        return float('inf')
    
    avg_sojourn, _, _ = simulate_queue(mu)
    penalty = c * (mu**2)
    return avg_sojourn + penalty

def optimize(budget=1000, c=0.1, lambda_=1.5):
    # Start with analytical solution approximation
    # Using numerical solution of cubic equation
    mu = max(lambda_ + 1, 3.0)  # Initial guess
    
    best_mu = mu
    best_obj = objective(mu, c, lambda_)
    
    step_size = 0.1
    evaluations = 1
    
    while evaluations < budget:
        # Try both directions
        for direction in [-1, 1]:
            if evaluations >= budget:
                break
                
            test_mu = mu + direction * step_size
            if test_mu <= lambda_:
                continue
                
            obj = objective(test_mu, c, lambda_)
            evaluations += 1
            
            if obj < best_obj:
                best_obj = obj
                best_mu = test_mu
                mu = test_mu
                break
        else:
            step_size *= 0.5
        
        if step_size < 1e-4:
            break
    
    return best_mu, best_obj, evaluations

# Run optimization
mu_opt, obj_opt, evals = optimize(budget=1000)
sojourn, waiting, frac = simulate_queue(mu_opt)

print(f"Optimized solution:")
print(f"mu = {mu_opt:.3f}")
print(f"Average sojourn time = {sojourn:.3f}")
print(f"Penalty cost = {0.1 * mu_opt**2:.3f}")
print(f"Total objective = {obj_opt:.3f}")
print(f"Evaluations used = {evals}")

# run this 1000 times and find the mean and std deviation for the total objective
num_trials = 1000
total_objectives = []
for _ in range(num_trials):
    mu_opt, obj_opt, evals = optimize(budget=1000)
    sojourn, waiting, frac = simulate_queue(mu_opt)
    penalty = 0.1 * (mu_opt**2)
    total_objective = sojourn + penalty
    total_objectives.append(total_objective)
mean_total_objective = np.mean(total_objectives)
std_total_objective = np.std(total_objectives)
print(f"Mean of total objectives over {num_trials} trials: {mean_total_objective:.4f}")
print(f"Standard deviation of total objectives over {num_trials} trials: {std_total_objective:.4f}")

Optimized solution:
mu = 2.728
Average sojourn time = 0.766
Penalty cost = 0.744
Total objective = 1.510
Evaluations used = 32
Mean of total objectives over 1000 trials: 1.5104
Standard deviation of total objectives over 1000 trials: 0.0000
